In [3]:
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.dialects.mysql import TEXT, LONGTEXT  # 데이터 타입을 직접 지정하기 위해 임포트


In [6]:
"""
JSON 파일을 읽어 MySQL 데이터베이스에 저장하는 전체 과정을 수행합니다.
사용자 설정, 데이터베이스 생성, 데이터 로딩, 타입 변환, 저장을 포함합니다.
"""
# --- 1. 사용자 설정 부분 ---
# 사용자의 환경에 맞게 아래 정보들을 수정해주세요.

# 1-1. 불러올 JSON 파일의 경로
# 각 줄이 하나의 JSON 객체인 .jsonl 형태이거나, JSON 배열 형태의 파일이어야 합니다.
json_file_path = 'steam_games_structured_data.json'

# 1-2. 접속할 MySQL 데이터베이스 정보
db_connection_info = {
    'host': 'localhost',
    'port': 3306,
    'user': 'test1',
    'password': 'test1',
    # 이 이름의 데이터베이스가 없으면 자동으로 생성됩니다.
    'database': 'steam_structured_db'
}

# 1-3. 저장할 테이블 이름
mysql_table_name = 'steam_structured_data'

In [7]:
# --- 2. 데이터베이스 준비 ---
db_info = db_connection_info
# 데이터베이스가 없는 경우를 대비해, DB 이름을 제외한 초기 연결 URL 생성
initial_db_url = f"mysql+pymysql://{db_info['user']}:{db_info['password']}@{db_info['host']}:{db_info['port']}"
engine_no_db = create_engine(initial_db_url)

with engine_no_db.connect() as connection:
    print(f"'{db_info['database']}' 데이터베이스 존재 여부 확인 및 생성 시도...")
    # MySQL은 CREATE DATABASE IF NOT EXISTS를 지원합니다.
    connection.execute(text(f"CREATE DATABASE IF NOT EXISTS {db_info['database']} CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci"))
    print(f"'{db_info['database']}' 데이터베이스가 준비되었습니다.")

# 실제 작업을 위한 최종 데이터베이스 연결 URL
db_url = f"{initial_db_url}/{db_info['database']}"
engine = create_engine(db_url)
print("데이터베이스 엔진이 성공적으로 생성되었습니다.")

# --- 3. JSON 파일 읽기 ---
print(f"\n'{json_file_path}' 파일 읽기를 시도합니다...")
# JSON Lines 파일(.jsonl)인 경우 lines=True 옵션을 사용합니다.
# 일반적인 JSON 배열 파일인 경우 이 옵션을 제거하면 됩니다.
try:
    df = pd.read_json(json_file_path, lines=True)
except ValueError:
    print("lines=True로 읽기 실패. 일반 JSON 배열 형식으로 다시 시도합니다.")
    df = pd.read_json(json_file_path)

print("[원본 DataFrame (상위 5개)]")
print(df.head())

'steam_structured_db' 데이터베이스 존재 여부 확인 및 생성 시도...
'steam_structured_db' 데이터베이스가 준비되었습니다.
데이터베이스 엔진이 성공적으로 생성되었습니다.

'steam_games_structured_data.json' 파일 읽기를 시도합니다...
lines=True로 읽기 실패. 일반 JSON 배열 형식으로 다시 시도합니다.
[원본 DataFrame (상위 5개)]
     appid                   name  peak_ccu  required_age  price  dlc_count  \
0    20200       Galactic Bowling         0             0  19.99          0   
1   655370           Train Bandit         0             0   0.99          0   
2  1732930           Jolt Project         0             0   4.99          0   
3  1355720               Henosis™         0             0   5.99          0   
4  1139950  Two Weeks in Painland         0             0   0.00          0   

   windows    mac  linux  metacritic_score  user_score  positive  negative  \
0     True  False  False                 0           0         6        11   
1     True   True  False                 0           0        53         5   
2     True  False  False                 0           0   

In [8]:
# --- 4. JSON 형식의 열을 문자열로 변환 ---
# DataFrame을 순회하며 list나 dict 타입의 데이터를 가진 열을 찾습니다.
dtype_mapping = {}
columns_to_convert = []

for col in df.columns:
    # 첫 번째 non-null 값을 확인하여 타입 검사
    first_valid_index = df[col].first_valid_index()
    if first_valid_index is not None:
        value = df[col].loc[first_valid_index]
        if isinstance(value, (dict, list)):
            columns_to_convert.append(col)

if columns_to_convert:
    print(f"\nJSON 문자열로 변환할 열: {columns_to_convert}")
    for col in columns_to_convert:
        # Na/NaN/None이 아닌 값에 대해서만 json.dumps 적용
        df[col] = df[col].apply(lambda x: json.dumps(x, ensure_ascii=False) if pd.notna(x) and isinstance(x, (dict, list)) else None)
        dtype_mapping[col] = TEXT  # 모든 JSON 열을 TEXT 타입으로 매핑
    print("[JSON 열 변환 후 DataFrame (상위 5개)]")
    print(df.head())
else:
    print("\nJSON으로 변환할 list 또는 dict 타입의 열을 찾지 못했습니다.")


JSON으로 변환할 list 또는 dict 타입의 열을 찾지 못했습니다.


In [9]:
df

,appid,name,peak_ccu,required_age,price,dlc_count,windows,mac,linux,metacritic_score,user_score,positive,negative,score_rank,achievements,recommendations,average_playtime_forever,median_playtime_forever
0,20200,Galactic Bowling,0,0,19.99,0,True,False,False,0,0,6,11,,30,0,0,0
1,655370,Train Bandit,0,0,0.99,0,True,True,False,0,0,53,5,,12,0,0,0
2,1732930,Jolt Project,0,0,4.99,0,True,False,False,0,0,0,0,,0,0,0,0
3,1355720,Henosis™,0,0,5.99,0,True,True,True,0,0,3,0,,0,0,0,0
4,1139950,Two Weeks in Painland,0,0,0.00,0,True,True,False,0,0,50,8,,17,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111447,3600970,Paragon Of Time,0,0,2.99,0,True,False,False,0,0,5,0,,0,0,0,0
111448,3543710,A Few Days With : Hazel,0,0,2.69,0,True,False,False,0,0,0,0,,7,0,0,0
111449,3265370,MosGhost,0,0,7.99,0,True,False,False,0,0,24,12,,0,0,0,0
111450,3423620,AccuBow VR,0,0,0.00,0,True,False,False,0,0,0,0,,0,0,0,0


In [10]:
# --- 5. DataFrame을 MySQL에 저장 ---
print(f"\n'{mysql_table_name}' 테이블에 데이터 저장을 시도합니다...")
df.to_sql(
    name=mysql_table_name,
    con=engine,
    if_exists='replace',
    index=False,
    dtype=dtype_mapping if dtype_mapping else None
)
print(f"'{mysql_table_name}' 테이블에 데이터가 성공적으로 저장되었습니다.")

# --- 6. 검증: 저장된 데이터 다시 읽어오기 ---
print("\n[검증] 데이터베이스에서 데이터를 다시 읽어옵니다.")
with engine.connect() as connection:
    query = text(f"SELECT * FROM {mysql_table_name} LIMIT 5")  # 5개만 읽어와서 확인
    result_df = pd.read_sql(query, connection)

print("MySQL에서 읽어온 데이터 (상위 5개):")
print(result_df)
if columns_to_convert and not result_df.empty:
    # 변환된 열 중 첫 번째 열의 타입만 확인
    check_col = columns_to_convert[0]
    if check_col in result_df.columns:
         print(f"\n읽어온 데이터의 '{check_col}' 열 타입:", type(result_df[check_col][0]))


'steam_structured_data' 테이블에 데이터 저장을 시도합니다...
'steam_structured_data' 테이블에 데이터가 성공적으로 저장되었습니다.

[검증] 데이터베이스에서 데이터를 다시 읽어옵니다.
MySQL에서 읽어온 데이터 (상위 5개):
     appid                   name  peak_ccu  required_age  price  dlc_count  \
0    20200       Galactic Bowling         0             0  19.99          0   
1   655370           Train Bandit         0             0   0.99          0   
2  1732930           Jolt Project         0             0   4.99          0   
3  1355720               Henosis™         0             0   5.99          0   
4  1139950  Two Weeks in Painland         0             0   0.00          0   

   windows  mac  linux  metacritic_score  user_score  positive  negative  \
0        1    0      0                 0           0         6        11   
1        1    1      0                 0           0        53         5   
2        1    0      0                 0           0         0         0   
3        1    1      1                 0           0         3        